In [1]:
from scipy.integrate import dblquad
import tensorflow as tf
import numpy as np
tfk=tf.keras
tfkl=tf.keras.layers


2023-10-24 10:46:49.559801: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-24 10:46:49.562189: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-24 10:46:49.593936: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-24 10:46:49.594603: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-24 10:46:50.222737: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
f=lambda x,y: x*y

In [3]:
f(0,0)

0

In [4]:
area = dblquad(f, 0, 0.5, lambda x: 0, lambda x: 1-2*x)
area

(0.010416666666666668, 4.101620128472366e-16)

In [5]:
def net(x:float,y:float):
    return x*y

In [6]:
area=dblquad(lambda x,y: net(x,y), 0, 0.5, lambda x: 0, lambda x: 1-2*x)
print(area)

(0.010416666666666668, 4.101620128472366e-16)


In [7]:
area=dblquad(net, 0, 0.5, lambda x: 0, lambda x: 1-2*x)
print(area)

(0.010416666666666668, 4.101620128472366e-16)


In [21]:

#net that takes as the one one we will use 
input_shape=(2,)
seed=42
input_layer = tfkl.Input(shape=input_shape, name='Input')
x1 = tfkl.Dense(2, activation=tf.math.sin, kernel_initializer = tfk.initializers.GlorotUniform(seed),bias_initializer=tfk.initializers.GlorotUniform(seed))(input_layer)
output_layer = tfkl.Dense(1, activation='linear', kernel_initializer = tfk.initializers.GlorotUniform(seed),bias_initializer=tfk.initializers.GlorotUniform(seed))(x1)

model = tf.keras.Model(input_layer, output_layer)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 2)]               0         
                                                                 
 dense_6 (Dense)             (None, 2)                 6         
                                                                 
 dense_7 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9 (36.00 Byte)
Trainable params: 9 (36.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
x=tf.constant([[0.0,0.0],[1.0,1.0]])
print(x.shape)

#the input should be presented in batches so shape (2,) ->(1,2 ) if you have n points (n,2)


#input (n,2) output (n,1) in our case 


out_tf=model(x)
out_predict=model.predict(x)

print()


print('model(input)')
print(type(out_tf))
print(out_tf)

print()

print('model.predict(input)')
print(type(out_predict))
print(out_predict)


print()

print('if i squeeze a tensor')
print(tf.squeeze(model(x)))

(2, 2)
1/1 [==============================] - 0s 31ms/step

model(input)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[[-0.44101208]
 [-0.4341511 ]], shape=(2, 1), dtype=float32)

model.predict(input)
<class 'numpy.ndarray'>
[[-0.44101208]
 [-0.4341511 ]]

if i squeeze a tensor
tf.Tensor([-0.44101208 -0.4341511 ], shape=(2,), dtype=float32)


In [23]:
def eval_net(x:float,y:float):
    p=tf.constant([[x,y]])
    return tf.squeeze(model(p))

In [24]:
area=dblquad(lambda x,y: eval_net(x,y), 0, 0.5, lambda x: 0, lambda x: 1-2*x)
print(area)


(-0.09973668300717066, 9.29645908202773e-09)


In [25]:
print(model.trainable_variables)

[<tf.Variable 'dense_6/kernel:0' shape=(2, 2) dtype=float32, numpy=
array([[-0.5329278 , -0.00387776],
       [ 0.5277635 , -1.1752602 ]], dtype=float32)>, <tf.Variable 'dense_6/bias:0' shape=(2,) dtype=float32, numpy=array([-0.5329278 , -0.00387776], dtype=float32)>, <tf.Variable 'dense_7/kernel:0' shape=(2, 1) dtype=float32, numpy=
array([[-0.61537194],
       [-0.00447774]], dtype=float32)>, <tf.Variable 'dense_7/bias:0' shape=(1,) dtype=float32, numpy=array([-0.7536737], dtype=float32)>]


In [26]:
def F(x,y):
    return -0.61537194*tf.math.sin((-0.5329278*x)+(0.5277635*y)-0.5329278)  -0.00447774*tf.math.sin((-0.00387776*x)+(-1.1752602*y)-0.00387776) -0.7536737



F(1.0,1.0)


<tf.Tensor: shape=(), dtype=float32, numpy=-0.43415114>

In [27]:
area=dblquad(F, 0, 0.5, lambda x: 0, lambda x: 1-2*x)
print(area)

(-0.09973668498360751, 1.0804009353264507e-08)


In [33]:
points=tf.constant([[1.0,1.0]])

with tf.GradientTape() as tape:
    tape.watch(points)
    y=model(points)
dy=tape.gradient(y,points)


print(dy)



points=tf.constant([[1.0,1.0]])

with tf.GradientTape() as tape:
    tape.watch(points)
    y=model(points)
dy=tape.gradient(y,points)


print(dy)


#if you eval the gradient on a set of points the grad will have size (number of points,2)

tf.Tensor([[ 0.28161228 -0.2768869 ]], shape=(1, 2), dtype=float32)
